<a href="https://colab.research.google.com/github/prinaldi3/Denoising/blob/main/pattern_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.append('/home/pmm/Development/xca')
from xca.data_synthesis.builder import cycle_params
from pathlib import Path
from xca.examples.arxiv200800283.example_synthesis import log_reflections
import numpy as np
import os
import json
import collections


# Parameters
Some parameters are randomized by default by the function args in cycle_params. Any other parameter can be cycled by feeding a corresponding keyword arg and tuple of (min,max). 

Adding constant shift to background as option. 

In [ ]:
# Number of profiles per structure
n_profs = 10
out_dir = './'
cif_paths = list(Path('../../refined_cifs/').glob("*.cif"))
output_path = Path('./xarrays')
output_path.mkdir(parents=True, exist_ok=True)
xr_path = output_path
tfr_path = "BTO_sim.tfrecord"
param_dict = {'wavelength': 0.1671,
              'noise_std' :0.,
             'instrument_radius' : 1065.8822732979447,
             'theta_m' : 0.0,
             'tth_min' : 0.01,
             'tth_max' : 12.0,
             'n_datapoints' : 1750}
kwargs = {'bkg_1': (-1e-4,1e-4),
          'bkg_0': (0, 1e-3)}




In [ ]:
mapping = {cif.stem:i for i, cif in enumerate(cif_paths)}
print(mapping)

In [ ]:
for cif in cif_paths:
    print(cif)
    phase = cif.stem
    param_dict['input_cif'] = cif
    cycle_params(n_profs,
                 output_path,
                 input_params=param_dict, 
                 march_range=(0.999,1),
                 sample_height=(-0.01,0.01),
                 shape_limit=0.002,
                 preferred_axes=None,
                 n_jobs=58, 
                 **kwargs)

In [ ]:
from xca.ml.tf_data_proc import xr_dir_to_record, _int64_feature
xr_dir_to_record(xr_path, tfr_path, "input_cif",
                transform = lambda x: _int64_feature(mapping[x]),
                shuffle=True)

In [ ]:
path = list(output_path.glob("*.nc"))[0]

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
da = xr.open_dataarray(path)

In [ ]:
da

In [ ]:
np.max(da.coords["2theta"].data)

In [ ]:
da.plot()